# DATABRICKS #1 MountS3 & Create DataFrames
Reminder: This notebook is to be run inside of DATABRICKS.

#### Import

In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

#### Read CSV

In [ ]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

#### Get AWS Access key & Secret key

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

#### MOUNT S3 DATABASE

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e2a0bfcc015-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e2a0bfcc015-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

#### Check if Database was mounted

In [ ]:
display(dbutils.fs.ls("/mnt/user-0e2a0bfcc015-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.geo/,0e2a0bfcc015.geo/,0,1706710075083
dbfs:/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.pin/,0e2a0bfcc015.pin/,0,1706710075083
dbfs:/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.user/,0e2a0bfcc015.user/,0,1706710075083


In [ ]:
%sql
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


## DataFrame for pinterest post data

In [ ]:
# Pinterest Post Data

# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_pin = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.pin/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest post data
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_pin)

# Display Spark dataframe to check its content
display(df_pin)

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
event-planning,Το όνομα που επέλεξε η μαμά Ανδριανή για τη γλυκιά Τιτίκα δεν είναι καθόλου τυχαίο. Και φυσικά δεν άφησε τίποτα στην τύχη ούτε την ημέρα της βάπτισης. Ανέθεσε την οργάνωση στην…,1,4,https://i.pinimg.com/originals/db/aa/d2/dbaad28fa85012a4ea6958540d98a8e5.jpg,4387,image,Manosbojana Katsareas,Local save in /data/event-planning,"Diy Flowers,Flower Diy,Baptism Decorations,Christening,Event Planning,Wedding Planner,Baptism Ideas,Birthday,Party",Βάπτιση: H παραμυθένια βάπτιση της Τιτίκας με θέμα το μονόκερο από την e.m. for you,ae5e7377-f1bd-4ac5-94de-bee317f51a43
home-decor,"Традиционные шведские коттеджи, обычно с красным фасадом — это настоящее воплощением идеального зимнего уюта. Они обычно оформлены очень просто и ✌PUFIK. Beautiful Interiors. On…",1,136k,https://i.pinimg.com/originals/32/eb/72/32eb72e4fd8654c115a64528bd1f34b4.png,6717,image,PUFIK Interiors & Inspirations,Local save in /data/home-decor,"Scandinavian Cottage,Swedish Cottage,Swedish Home Decor,Swedish Farmhouse,Swedish Style,Swedish Kitchen,Kitchen Black,Swedish House,Cozy Cottage",〚 Уютные шведские коттеджи от Carina Olander 〛 ◾ Фото ◾ Идеи ◾ Дизайн,bc5ab9ee-505e-44f6-92ba-677fe4fdf3e3
home-decor,"6,636 Likes, 141 Comments - The Cottage Journal (@thecottagejournal) on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are g…",1,394,https://i.pinimg.com/originals/8c/17/a2/8c17a257b70780480bb89c3699363144.jpg,6633,image,Sarah Martin,Local save in /data/home-decor,"Diy Kitchen Cabinets,Kitchen Redo,Home Decor Kitchen,New Kitchen,Home Kitchens,Kitchen Remodeling,Aqua Kitchen,Kitchen Counters,Kitchen Islands",The Cottage Journal on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are giving. If you could paint your cabinets any…”,d136f6bc-840d-44f8-bbad-115eb7e6c51e
christmas,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",1,5k,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,1706,image,Wear24-7,Local save in /data/christmas,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,b5c8a1b5-9e90-4522-9bec-2477b698d5b7
christmas,"❤️ ❤️ MERRY CHRISTMAS ❤️ ❤️ ❤️ ❤️ Early Christmas Special:Buy 3 Get 1 Free, Buy 5 Get 2 Free,Deadline November 25. Color:GreenMaterial:Polyvinyl ChlorideItem Dimensions:LxWxH 20…",1,784,https://i.pinimg.com/originals/ef/40/7e/ef407e9568aa46fed4162bd1fd28786e.jpg,1676,image,paupoo,Local save in /data/christmas,"Christmas Hanging Baskets,Christmas Plants,Christmas Wreaths,Christmas Ornaments,Merry Christmas,Christmas Sale,Christmas Porch Ideas,Hanging Christmas Lights,Christmas Island",PAUPOO™ Pre-lit Artificial Christmas Hanging Basket - Flocked with Mixed Decorations and White LED Lights - Frosted Berry BUY 5 GET 2 FREE(7PACKS),3ed92c2d-9cca-4ccf-ac25-44a9d8bec919
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating

## DataFrame for pinterest geo data

In [ ]:
# Pinterest Geo Data

# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_geo = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.geo/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest geo data
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_geo)

# Display Spark dataframe to check its content
display(df_geo)

country,ind,latitude,longitude,timestamp
British Indian Ocean Territory (Chagos Archipelago),9455,-82.9272,-150.346,2022-03-15 01:46:32
British Indian Ocean Territory (Chagos Archipelago),6814,-86.5675,-149.565,2022-09-02 11:34:28
British Indian Ocean Territory (Chagos Archipelago),5111,-83.7472,8.65953,2021-04-01 00:56:57
Antarctica (the territory South of 60 deg S),10073,-32.8885,-170.295,2021-06-29 19:56:04
Antarctica (the territory South of 60 deg S),10073,-32.8885,-170.295,2021-06-29 19:56:04
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27 11:30:59
Antarctica (the territory South of 60 deg S),5162,-71.6607,-149.206,2019-09-27 19:06:43
Antarctica (the territory South of 60 deg S),1335,-77.9931,-175.682,2022-03-19 17:29:42
Antarctica (the territory South of 60 deg S),9185,-10.3764,-22.9809,2019-10-06 18:12:55


## DataFrame for pinterest user data

In [ ]:
# Pinterest User Data

# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_user = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.user/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest user data
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_user)

# Display Spark dataframe to check its content
display(df_user)

age,date_joined,first_name,ind,last_name
42,2017-02-18 00:31:22,Christopher,6353,Hernandez
27,2016-03-08 13:38:37,Christopher,2015,Bradshaw
59,2017-05-12 21:22:17,Alexander,10673,Cervantes
48,2016-02-27 16:57:44,Christopher,1857,Hamilton
45,2016-09-15 06:02:53,Christopher,10020,Hawkins
35,2015-10-22 22:42:23,Christopher,2041,Campbell
48,2016-06-13 17:09:14,Christopher,7031,Anderson
39,2016-06-29 20:43:59,Christina,6398,Davenport
20,2015-10-23 04:13:23,Alexandria,3599,Alvarado
20,2015-10-23 04:13:23,Alexandria,4256,Alvarado


You can unmount the DataFrame by using the below code.\
```dbutils.fs.unmount("/mnt/mount_name")```
